In [4]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from zipfile import ZipFile
from datetime import date, timedelta, datetime

In [5]:
print(str(datetime.now()) + " Running 6_COVID_14day_Metrics_Process")
yearDate = date.today().strftime('%m%d%y')
print("today: "+yearDate)
yearDate_long = date.today().strftime('%m/%d/%y')
print(yearDate_long)
yesterday = date.today() - timedelta(days=1)
yearDate_yesterday = yesterday.strftime('%m%d%y')
print("yesterday: "+yearDate_yesterday)

2020-11-15 12:55:23.007717 Running 6_COVID_14day_Metrics_Process
today: 111520
11/15/20
yesterday: 111420


In [6]:
covid_cases_file = "data/Florida_COVID19_Cases_by_County{}.xlsx".format(yearDate)
df_covid_cases = pd.read_excel(covid_cases_file, sheet_name = 'Florida_COVID19_Cases_by_County')
df_covid_case = df_covid_cases.drop(columns=['Unnamed: 0'],axis=1)
df_covid_case.head()

,OBJECTID,OBJECTID_12_13,DEPCODE,COUNTY,COUNTYNAME,County_1,State,PUIsTotal,Age_0_4,Age_5_14,...,EverMon,MonNow,Shape__Area,Shape__Length,GlobalID,NewPos,NewNeg,NewTested,NewPercPos,MedianAge
0,66,66,1,1,ALACHUA,Alachua,FL,26550,414,684,...,1769,1746,2.510756e+09,272135.471452,9eddb8fb-00dd-45f1-a09d-8e689049e28a,17,415,432,0.039352,24
1,17,17,2,3,BAKER,Baker,FL,3152,71,154,...,2221,2220,1.520246e+09,183737.354704,393e3bc9-079e-4ad6-9162-d77ee72feb59,5,27,32,0.156250,41
2,15,15,3,5,BAY,Bay,FL,11046,139,339,...,2028,2021,1.930771e+09,558412.990826,a66f3b06-8ee0-4dec-843e-f41d712aad2f,61,394,455,0.134066,42
3,40,40,4,7,BRADFORD,Bradford,FL,2968,69,160,...,1894,1863,7.767446e+08,167178.838253,588f1d06-056b-4af5-a2e7-a5366c833ba5,1,111,112,0.008929,46
4,42,42,5,9,BREVARD,Brevard,FL,32431,415,998,...,3251,3193,2.687644e+09,793218.333609,943db4a8-472c-444e-b81a-0e7cf9758ca2,145,934,1079,0.134384,44


In [7]:
df_new_case = df_covid_cases[['County_1','NewPos']].copy()
df_new_case.head()

,County_1,NewPos
0,Alachua,17
1,Baker,5
2,Bay,61
3,Bradford,1
4,Brevard,145


In [8]:
df_new_tested = df_covid_cases[['County_1','NewTested']].copy()
df_new_tested.head()

,County_1,NewTested
0,Alachua,432
1,Baker,32
2,Bay,455
3,Bradford,112
4,Brevard,1079


In [9]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [10]:
hosp_file = os.path.join(result_folder,"FL_County_Hospital_ICU{}.xlsx".format(yearDate))
df_hosps = pd.read_excel(hosp_file, sheet_name = 'Sheet1')
df_hosp = df_hosps.drop(columns=['Unnamed: 0'],axis=1)
df_hosp.head()

,County,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Available Pediatric ICU,Available Pediatric ICU%,Pediatric ICU Census,Total AdultICU Capacity,Total PediatricICU Capacity,Available,Available Capacity,Bed Census,Total Staffed Bed Capacity,COVID Hospitalizations
0,ALACHUA,276,34,0.109677,8,0.170213,39,310,47,237,0.136285,1502,1739,72
1,BAKER,0,0,0.000000,0,0.000000,0,0,0,116,0.185600,509,625,0
2,BAY,54,0,0.000000,4,1.000000,0,54,4,96,0.176796,447,543,23
3,BRADFORD,0,0,0.000000,0,0.000000,0,0,0,5,0.500000,5,10,0
4,BREVARD,120,47,0.281437,0,0.000000,0,167,0,528,0.323728,1103,1631,73


In [11]:
df_new_hosp = df_hosp[['County', 'COVID Hospitalizations']].copy()
df_new_hosp.head()

,County,COVID Hospitalizations
0,ALACHUA,72
1,BAKER,0
2,BAY,23
3,BRADFORD,0
4,BREVARD,73


In [12]:
covid_update_file = os.path.join('results/{}'.format(yearDate_yesterday),"COVID_daily_update{}.xlsx".format(yearDate_yesterday))
df_NewPositives = pd.read_excel(covid_update_file, sheet_name = 'NewPositive')
df_NewTesteds = pd.read_excel(covid_update_file, sheet_name = 'NewTested')
df_Hospitalizations = pd.read_excel(covid_update_file, sheet_name = 'Hospitalization')

In [13]:
# remove first unnamed column
df_Hospitalization = df_Hospitalizations.drop(columns=['Unnamed: 0'],axis=1)
df_NewPositive = df_NewPositives.drop(columns=['Unnamed: 0'],axis=1)
df_NewTested = df_NewTesteds.drop(columns=['Unnamed: 0'],axis=1)

In [14]:
df_NewPositive.head()

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/05/2020,11/06/2020,11/07/2020,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20
0,1,1,ALACHUA,Alachua,114,86,146,160,176,172,...,74,48,24,43,130,75,91,94,75,17
1,2,3,BAKER,Baker,10,9,11,20,15,13,...,13,4,2,5,5,12,4,8,9,5
2,3,5,BAY,Bay,13,27,12,21,14,37,...,45,27,19,30,80,58,67,36,51,61
3,4,7,BRADFORD,Bradford,4,6,18,23,15,17,...,8,4,1,7,4,8,10,3,2,1
4,5,9,BREVARD,Brevard,36,57,47,95,72,90,...,122,109,145,111,96,148,172,70,145,145


In [15]:
df_NewPositive_merge = pd.merge(df_NewPositive, df_new_case, left_on=['County_1'],right_on= ['County_1'], how = 'outer')
df_NewPositive_new = df_NewPositive_merge.rename(columns={"NewPos":yearDate_long})
df_NewPositive_new.head()

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/06/2020,11/07/2020,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20
0,1,1,ALACHUA,Alachua,114,86,146,160,176,172,...,48,24,43,130,75,91,94,75,17,17
1,2,3,BAKER,Baker,10,9,11,20,15,13,...,4,2,5,5,12,4,8,9,5,5
2,3,5,BAY,Bay,13,27,12,21,14,37,...,27,19,30,80,58,67,36,51,61,61
3,4,7,BRADFORD,Bradford,4,6,18,23,15,17,...,4,1,7,4,8,10,3,2,1,1
4,5,9,BREVARD,Brevard,36,57,47,95,72,90,...,109,145,111,96,148,172,70,145,145,145


In [16]:
df_NewTested_merge = pd.merge(df_NewTested, df_new_tested, left_on=['County_1'],right_on= ['County_1'], how = 'outer')
df_NewTested_new = df_NewTested_merge.rename(columns={"NewTested":yearDate_long})
df_NewTested_new.head()

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/06/2020,11/07/2020,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20
0,1,1,ALACHUA,Alachua,955,1290,1016,1426,2055,2202,...,1516,684,614,3646,1121,1901,1787,1158,432,432
1,2,3,BAKER,Baker,54,60,68,67,235,124,...,82,36,101,98,178,119,82,60,32,32
2,3,5,BAY,Bay,194,93,273,314,167,359,...,307,319,352,211,771,623,587,463,455,455
3,4,7,BRADFORD,Bradford,63,73,75,96,114,143,...,65,11,51,49,69,127,40,71,112,112
4,5,9,BREVARD,Brevard,1363,1133,671,1440,1806,1579,...,1153,1335,1923,1107,1342,1288,2115,1457,1079,1079


In [17]:
df_Hospitalization_merge = pd.merge(df_Hospitalization, df_new_hosp, left_on=['County'],right_on= ['County'], how='outer')
df_Hospitalization_new = df_Hospitalization_merge.rename(columns={"COVID Hospitalizations":yearDate_long})
df_Hospitalization_new.head()

,ID,County,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,2020-09-20 00:00:00,2020-09-21 00:00:00,...,11/06/2020,11/07/2020,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20
0,0,ALACHUA,87,83,89,77,68,70,69,72,...,61,64,69,71,73,74,74,69,70,72
1,1,BAKER,21,12,12,15,16,16,16,13,...,1,0,1,1,1,0,1,1,1,0
2,2,BAY,24,25,26,29,27,27,28,24,...,18,18,19,19,23,22,31,28,29,23
3,3,BRADFORD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,BREVARD,58,53,46,42,44,50,46,43,...,69,72,71,73,81,72,74,78,72,73


In [18]:
# Export updated covid data to excel file
print("Saving COVID_daily_update{}.xlsx in results/".format(yearDate))
with pd.ExcelWriter(os.path.join(result_folder,'COVID_daily_update{}.xlsx'.format(yearDate))) as writer:  
    df_NewPositive_new.to_excel(writer, sheet_name='NewPositive')
    df_NewTested_new.to_excel(writer, sheet_name='NewTested')
    df_Hospitalization_new.to_excel(writer, sheet_name='Hospitalization')

Saving COVID_daily_update111520.xlsx in results/


In [19]:
df_NewPositive_new['7dayPos'] = df_NewPositive_new.iloc[:,-7:].sum(axis=1)
df_NewPositive_new['14dayPos'] = df_NewPositive_new.iloc[:,-15:-1].sum(axis=1)
df_NewPositive_new

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,7dayPos,14dayPos
0,1,1,ALACHUA,Alachua,114,86,146,160,176,172,...,43,130,75,91,94,75,17,17,499,864
1,2,3,BAKER,Baker,10,9,11,20,15,13,...,5,5,12,4,8,9,5,5,48,92
2,3,5,BAY,Bay,13,27,12,21,14,37,...,30,80,58,67,36,51,61,61,414,637
3,4,7,BRADFORD,Bradford,4,6,18,23,15,17,...,7,4,8,10,3,2,1,1,29,54
4,5,9,BREVARD,Brevard,36,57,47,95,72,90,...,111,96,148,172,70,145,145,145,921,1758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,129,WAKULLA,Wakulla,19,14,8,12,8,14,...,12,1,20,12,11,15,3,3,65,110
65,66,131,WALTON,Walton,17,9,14,5,3,8,...,61,16,22,20,34,39,18,18,167,635
66,67,133,WASHINGTON,Washington,0,2,3,0,1,3,...,4,9,8,1,2,22,13,13,68,113
67,998,998,A State,State,1827,2701,2340,3309,3239,3603,...,6641,3876,4316,5719,5534,6787,4452,4452,35136,69238


In [20]:
df_NewTested_new['7dayTested'] = df_NewTested_new.iloc[:,-7:].sum(axis=1)
df_NewTested_new['14dayTested'] = df_NewTested_new.iloc[:,-15:-1].sum(axis=1)
df_NewTested_new

,DEPCODE,COUNTY,COUNTYNAME,County_1,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,...,11/08/2020,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,7dayTested,14dayTested
0,1,1,ALACHUA,Alachua,955,1290,1016,1426,2055,2202,...,614,3646,1121,1901,1787,1158,432,432,10477,17728
1,2,3,BAKER,Baker,54,60,68,67,235,124,...,101,98,178,119,82,60,32,32,601,1191
2,3,5,BAY,Bay,194,93,273,314,167,359,...,352,211,771,623,587,463,455,455,3565,6311
3,4,7,BRADFORD,Bradford,63,73,75,96,114,143,...,51,49,69,127,40,71,112,112,580,983
4,5,9,BREVARD,Brevard,1363,1133,671,1440,1806,1579,...,1923,1107,1342,1288,2115,1457,1079,1079,9467,18776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,129,WAKULLA,Wakulla,126,76,58,198,129,208,...,239,28,154,167,148,158,66,66,787,1604
65,66,131,WALTON,Walton,114,112,148,144,142,203,...,457,104,195,175,225,163,74,74,1010,3660
66,67,133,WASHINGTON,Washington,75,26,25,67,18,52,...,35,13,42,23,32,52,24,24,210,649
67,998,998,A State,State,47014,64194,52537,74522,77646,79780,...,106856,48365,52115,73420,75557,85600,45072,45072,425201,927262


In [21]:
df_Hospitalization_new.iloc[:,-1]

0       72
1        0
2       23
3        0
4       73
5      284
6        0
7       45
8       40
9       28
10      63
11      14
12       7
13     189
14      76
15       9
16       0
17       0
18       0
19       0
20       0
21      64
22      72
23     219
24       1
25      12
26      21
27      40
28      74
29      35
30       0
31      33
32      47
33       8
34     437
35       9
36       3
37      40
38       5
39     162
40      83
41     221
42      73
43     155
44      96
45       9
46      14
47      86
48      43
49       6
50      39
51      12
52       0
53       0
54       2
55      61
56       7
57       6
58    3118
Name: 11/15/20, dtype: int64

In [22]:
df_Hospitalization_new['NewHosp14AVE'] = (df_Hospitalization_new.iloc[:,-1] - df_Hospitalization_new.iloc[:,-15])/14
df_Hospitalization_new['Hosp14AVE'] = df_Hospitalization_new.iloc[:,-15:-1].mean(axis=1)
df_Hospitalization_new['Hosp7AVE'] = df_Hospitalization_new.iloc[:,-9:-2].mean(axis=1)
df_Hospitalization_new.head()

,ID,County,2020-09-14 00:00:00,2020-09-15 00:00:00,2020-09-16 00:00:00,2020-09-17 00:00:00,2020-09-18 00:00:00,2020-09-19 00:00:00,2020-09-20 00:00:00,2020-09-21 00:00:00,...,11/09/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,NewHosp14AVE,Hosp14AVE,Hosp7AVE
0,0,ALACHUA,87,83,89,77,68,70,69,72,...,71,73,74,74,69,70,72,0.357143,67.357143,71.857143
1,1,BAKER,21,12,12,15,16,16,16,13,...,1,1,0,1,1,1,0,-0.285714,1.214286,0.714286
2,2,BAY,24,25,26,29,27,27,28,24,...,19,23,22,31,28,29,23,0.142857,21.214286,25.000000
3,3,BRADFORD,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,0.000000,0.000000
4,4,BREVARD,58,53,46,42,44,50,46,43,...,73,81,72,74,78,72,73,0.142857,71.428571,74.714286


In [23]:
df_case_merge = pd.merge(df_NewPositive_new,df_NewTested_new, left_on = ['County_1'], right_on = ['County_1'], how="outer")
df_case_merge

,DEPCODE_x,COUNTY_x,COUNTYNAME_x,County_1,2020-09-14 00:00:00_x,2020-09-15 00:00:00_x,2020-09-16 00:00:00_x,2020-09-17 00:00:00_x,2020-09-18 00:00:00_x,2020-09-19 00:00:00_x,...,11/08/2020_y,11/09/20_y,11/10/20_y,11/11/20_y,11/12/20_y,11/13/20_y,11/14/20_y,11/15/20_y,7dayTested,14dayTested
0,1,1,ALACHUA,Alachua,114,86,146,160,176,172,...,614,3646,1121,1901,1787,1158,432,432,10477,17728
1,2,3,BAKER,Baker,10,9,11,20,15,13,...,101,98,178,119,82,60,32,32,601,1191
2,3,5,BAY,Bay,13,27,12,21,14,37,...,352,211,771,623,587,463,455,455,3565,6311
3,4,7,BRADFORD,Bradford,4,6,18,23,15,17,...,51,49,69,127,40,71,112,112,580,983
4,5,9,BREVARD,Brevard,36,57,47,95,72,90,...,1923,1107,1342,1288,2115,1457,1079,1079,9467,18776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,129,WAKULLA,Wakulla,19,14,8,12,8,14,...,239,28,154,167,148,158,66,66,787,1604
65,66,131,WALTON,Walton,17,9,14,5,3,8,...,457,104,195,175,225,163,74,74,1010,3660
66,67,133,WASHINGTON,Washington,0,2,3,0,1,3,...,35,13,42,23,32,52,24,24,210,649
67,998,998,A State,State,1827,2701,2340,3309,3239,3603,...,106856,48365,52115,73420,75557,85600,45072,45072,425201,927262


In [24]:
df_case = df_case_merge[['DEPCODE_x','County_1','COUNTYNAME_x','7dayPos','14dayPos','7dayTested','14dayTested']].copy()
df_case

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested
0,1,Alachua,ALACHUA,499,864,10477,17728
1,2,Baker,BAKER,48,92,601,1191
2,3,Bay,BAY,414,637,3565,6311
3,4,Bradford,BRADFORD,29,54,580,983
4,5,Brevard,BREVARD,921,1758,9467,18776
...,...,...,...,...,...,...,...
64,65,Wakulla,WAKULLA,65,110,787,1604
65,66,Walton,WALTON,167,635,1010,3660
66,67,Washington,WASHINGTON,68,113,210,649
67,998,State,A State,35136,69238,425201,927262


In [25]:
df_case['7daysPRate']=df_case['7dayPos']/df_case['7dayTested']
df_case['14daysPRate']=df_case['14dayPos']/df_case['14dayTested']
df_case['WeekPR_Ratio']=df_case['7daysPRate']/((df_case['14dayPos']-df_case['7dayPos'])/(df_case['14dayTested']-df_case['7dayTested']))
df_case['WeekPosi_Ratio']=df_case['7dayPos']/(df_case['14dayPos']-df_case['7dayPos'])

In [26]:
df_case 

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio
0,1,Alachua,ALACHUA,499,864,10477,17728,0.047628,0.048736,0.946169,1.367123
1,2,Baker,BAKER,48,92,601,1191,0.079867,0.077246,1.070942,1.090909
2,3,Bay,BAY,414,637,3565,6311,0.116129,0.100935,1.430001,1.856502
3,4,Bradford,BRADFORD,29,54,580,983,0.050000,0.054934,0.806000,1.160000
4,5,Brevard,BREVARD,921,1758,9467,18776,0.097285,0.093630,1.081994,1.100358
...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,WAKULLA,65,110,787,1604,0.082592,0.068579,1.499506,1.444444
65,66,Walton,WALTON,167,635,1010,3660,0.165347,0.173497,0.936257,0.356838
66,67,Washington,WASHINGTON,68,113,210,649,0.323810,0.174114,3.158942,1.511111
67,998,State,A State,35136,69238,425201,927262,0.082634,0.074669,1.216563,1.030321


In [27]:
df_hosp_14 = df_Hospitalization_new[['County','NewHosp14AVE','Hosp14AVE','Hosp7AVE']].copy()
df_hosp_14new = df_hosp_14.astype({'NewHosp14AVE': 'int64','Hosp14AVE': 'int64'})
df_hosp_14new.head()

,County,NewHosp14AVE,Hosp14AVE,Hosp7AVE
0,ALACHUA,0,67,71.857143
1,BAKER,0,1,0.714286
2,BAY,0,21,25.000000
3,BRADFORD,0,0,0.000000
4,BREVARD,0,71,74.714286


In [28]:
a = set(df_case['COUNTYNAME_x'].values)
b = set(df_hosp_14new['County'].values)

In [29]:
b.difference(a)

{'MIAMI-DADE'}

In [30]:
df_hosp_14new.loc[df_hosp_14new['County']=='MIAMI-DADE','County']='DADE'

In [31]:
a = set(df_case['COUNTYNAME_x'].values)
b = set(df_hosp_14new['County'].values)
b.difference(a)

set()

In [32]:
df_merge = pd.merge(df_case, df_hosp_14new, left_on=['COUNTYNAME_x'],right_on= ['County'], how = 'left')
df_merge['WeekHosp_Ratio']=df_merge['Hosp7AVE']/(df_merge['Hosp14AVE']*14-df_merge['Hosp7AVE']*7)*7
df_merge.head(len(df_merge))

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio,County,NewHosp14AVE,Hosp14AVE,Hosp7AVE,WeekHosp_Ratio
0,1,Alachua,ALACHUA,499,864,10477,17728,0.047628,0.048736,0.946169,1.367123,ALACHUA,0.0,67.0,71.857143,1.156322
1,2,Baker,BAKER,48,92,601,1191,0.079867,0.077246,1.070942,1.090909,BAKER,0.0,1.0,0.714286,0.555556
2,3,Bay,BAY,414,637,3565,6311,0.116129,0.100935,1.430001,1.856502,BAY,0.0,21.0,25.000000,1.470588
3,4,Bradford,BRADFORD,29,54,580,983,0.050000,0.054934,0.806000,1.160000,BRADFORD,0.0,0.0,0.000000,NaN
4,5,Brevard,BREVARD,921,1758,9467,18776,0.097285,0.093630,1.081994,1.100358,BREVARD,0.0,71.0,74.714286,1.110403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,WAKULLA,65,110,787,1604,0.082592,0.068579,1.499506,1.444444,NaN,NaN,NaN,NaN,NaN
65,66,Walton,WALTON,167,635,1010,3660,0.165347,0.173497,0.936257,0.356838,WALTON,0.0,10.0,12.285714,1.592593
66,67,Washington,WASHINGTON,68,113,210,649,0.323810,0.174114,3.158942,1.511111,WASHINGTON,0.0,3.0,4.285714,2.500000
67,998,State,A State,35136,69238,425201,927262,0.082634,0.074669,1.216563,1.030321,A State,53.0,2817.0,3064.571429,1.192705


In [33]:
df_sort = df_merge.sort_values(by=['DEPCODE_x'])
df_sort['County']=df_sort['County'].str.title()
df_sort

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio,County,NewHosp14AVE,Hosp14AVE,Hosp7AVE,WeekHosp_Ratio
0,1,Alachua,ALACHUA,499,864,10477,17728,0.047628,0.048736,0.946169,1.367123,Alachua,0.0,67.0,71.857143,1.156322
1,2,Baker,BAKER,48,92,601,1191,0.079867,0.077246,1.070942,1.090909,Baker,0.0,1.0,0.714286,0.555556
2,3,Bay,BAY,414,637,3565,6311,0.116129,0.100935,1.430001,1.856502,Bay,0.0,21.0,25.000000,1.470588
3,4,Bradford,BRADFORD,29,54,580,983,0.050000,0.054934,0.806000,1.160000,Bradford,0.0,0.0,0.000000,NaN
4,5,Brevard,BREVARD,921,1758,9467,18776,0.097285,0.093630,1.081994,1.100358,Brevard,0.0,71.0,74.714286,1.110403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,WAKULLA,65,110,787,1604,0.082592,0.068579,1.499506,1.444444,NaN,NaN,NaN,NaN,NaN
65,66,Walton,WALTON,167,635,1010,3660,0.165347,0.173497,0.936257,0.356838,Walton,0.0,10.0,12.285714,1.592593
66,67,Washington,WASHINGTON,68,113,210,649,0.323810,0.174114,3.158942,1.511111,Washington,0.0,3.0,4.285714,2.500000
67,998,State,A State,35136,69238,425201,927262,0.082634,0.074669,1.216563,1.030321,A State,53.0,2817.0,3064.571429,1.192705


In [34]:
df_sort_drop_unknown = df_sort.drop([68])
df_sort_drop_unknown

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio,County,NewHosp14AVE,Hosp14AVE,Hosp7AVE,WeekHosp_Ratio
0,1,Alachua,ALACHUA,499,864,10477,17728,0.047628,0.048736,0.946169,1.367123,Alachua,0.0,67.0,71.857143,1.156322
1,2,Baker,BAKER,48,92,601,1191,0.079867,0.077246,1.070942,1.090909,Baker,0.0,1.0,0.714286,0.555556
2,3,Bay,BAY,414,637,3565,6311,0.116129,0.100935,1.430001,1.856502,Bay,0.0,21.0,25.000000,1.470588
3,4,Bradford,BRADFORD,29,54,580,983,0.050000,0.054934,0.806000,1.160000,Bradford,0.0,0.0,0.000000,NaN
4,5,Brevard,BREVARD,921,1758,9467,18776,0.097285,0.093630,1.081994,1.100358,Brevard,0.0,71.0,74.714286,1.110403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,VOLUSIA,544,1189,6678,14845,0.081462,0.080094,1.031467,0.843411,Volusia,0.0,59.0,61.428571,1.085859
64,65,Wakulla,WAKULLA,65,110,787,1604,0.082592,0.068579,1.499506,1.444444,NaN,NaN,NaN,NaN,NaN
65,66,Walton,WALTON,167,635,1010,3660,0.165347,0.173497,0.936257,0.356838,Walton,0.0,10.0,12.285714,1.592593
66,67,Washington,WASHINGTON,68,113,210,649,0.323810,0.174114,3.158942,1.511111,Washington,0.0,3.0,4.285714,2.500000


In [35]:
print("Saving COVID_14_update{}.xlsx in results/".format(yearDate))
df_sort_drop_unknown.to_excel(os.path.join(result_folder,"COVID_14_update{}.xlsx".format(yearDate)))

Saving COVID_14_update111520.xlsx in results/


In [36]:
gdf_county_polygon = gpd.read_file("shp/Florida_Counties.shp")
gdf_county_polygon.head()

,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,21,041,Gilchrist,2000-05-16,9.908353e+09,4.873000e+05,"POLYGON ((-82.65814 29.83028, -82.65803 29.822..."
1,2,54,107,Putnam,2000-05-16,2.305869e+10,7.629677e+05,"POLYGON ((-81.58084 29.83956, -81.57893 29.835..."
2,3,62,123,Taylor,2000-05-16,2.891747e+10,8.772527e+05,"MULTIPOLYGON (((-83.73037 30.30358, -83.70368 ..."
3,4,46,091,Okaloosa,2000-05-16,2.562159e+10,1.087058e+06,"MULTIPOLYGON (((-86.39159 30.64970, -86.39175 ..."
4,5,7,013,Calhoun,2000-05-16,1.604809e+10,6.313440e+05,"POLYGON ((-84.93266 30.60637, -84.93287 30.605..."


In [37]:
a = set(df_sort_drop_unknown['County_1'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'State'}

In [38]:
b.difference(a)

set()

In [39]:
df_merge_poly = pd.merge(df_sort_drop_unknown, gdf_county_polygon, left_on = ['County_1'],right_on= ['COUNTYNAME'], how = 'outer')
df_merge_poly.head(len(df_merge_poly))

,DEPCODE_x,County_1,COUNTYNAME_x,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,...,Hosp7AVE,WeekHosp_Ratio,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,Alachua,ALACHUA,499,864,10477,17728,0.047628,0.048736,0.946169,...,71.857143,1.156322,66.0,1.0,001,Alachua,2000-05-16,2.702324e+10,8.927266e+05,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,2,Baker,BAKER,48,92,601,1191,0.079867,0.077246,1.070942,...,0.714286,0.555556,17.0,2.0,003,Baker,2000-05-16,1.636216e+10,6.027173e+05,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,3,Bay,BAY,414,637,3565,6311,0.116129,0.100935,1.430001,...,25.000000,1.470588,15.0,3.0,005,Bay,2000-05-16,2.084232e+10,1.834701e+06,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,4,Bradford,BRADFORD,29,54,580,983,0.050000,0.054934,0.806000,...,0.000000,NaN,40.0,4.0,007,Bradford,2000-05-16,8.359863e+09,5.483267e+05,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,5,Brevard,BREVARD,921,1758,9467,18776,0.097285,0.093630,1.081994,...,74.714286,1.110403,42.0,5.0,009,Brevard,2000-05-16,2.893695e+10,2.600567e+06,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,VOLUSIA,544,1189,6678,14845,0.081462,0.080094,1.031467,...,61.428571,1.085859,25.0,64.0,127,Volusia,2000-05-16,3.364565e+10,1.837936e+06,"MULTIPOLYGON (((-81.09705 29.35180, -81.09292 ..."
64,65,Wakulla,WAKULLA,65,110,787,1604,0.082592,0.068579,1.499506,...,NaN,NaN,41.0,65.0,129,Wakulla,2000-05-16,1.702251e+10,9.167339e+05,"MULTIPOLYGON (((-84.41197 30.30244, -84.37858 ..."
65,66,Walton,WALTON,167,635,1010,3660,0.165347,0.173497,0.936257,...,12.285714,1.592593,51.0,66.0,131,Walton,2000-05-16,2.926099e+10,1.216747e+06,"POLYGON ((-86.03523 30.98825, -86.03532 30.982..."
66,67,Washington,WASHINGTON,68,113,210,649,0.323810,0.174114,3.158942,...,4.285714,2.500000,53.0,67.0,133,Washington,2000-05-16,1.721229e+10,8.010119e+05,"POLYGON ((-85.43543 30.56727, -85.46843 30.567..."


In [40]:
df_merge_poly_clean = df_merge_poly.drop(columns=['DEPCODE_x','COUNTYNAME_x','COUNTY','OBJECTID','ShapeSTAre','ShapeSTLen','DATESTAMP'])
df_merge_poly_clean

,County_1,7dayPos,14dayPos,7dayTested,14dayTested,7daysPRate,14daysPRate,WeekPR_Ratio,WeekPosi_Ratio,County,NewHosp14AVE,Hosp14AVE,Hosp7AVE,WeekHosp_Ratio,DEPCODE,COUNTYNAME,geometry
0,Alachua,499,864,10477,17728,0.047628,0.048736,0.946169,1.367123,Alachua,0.0,67.0,71.857143,1.156322,1.0,Alachua,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,Baker,48,92,601,1191,0.079867,0.077246,1.070942,1.090909,Baker,0.0,1.0,0.714286,0.555556,2.0,Baker,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,Bay,414,637,3565,6311,0.116129,0.100935,1.430001,1.856502,Bay,0.0,21.0,25.000000,1.470588,3.0,Bay,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,Bradford,29,54,580,983,0.050000,0.054934,0.806000,1.160000,Bradford,0.0,0.0,0.000000,NaN,4.0,Bradford,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,Brevard,921,1758,9467,18776,0.097285,0.093630,1.081994,1.100358,Brevard,0.0,71.0,74.714286,1.110403,5.0,Brevard,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Volusia,544,1189,6678,14845,0.081462,0.080094,1.031467,0.843411,Volusia,0.0,59.0,61.428571,1.085859,64.0,Volusia,"MULTIPOLYGON (((-81.09705 29.35180, -81.09292 ..."
64,Wakulla,65,110,787,1604,0.082592,0.068579,1.499506,1.444444,NaN,NaN,NaN,NaN,NaN,65.0,Wakulla,"MULTIPOLYGON (((-84.41197 30.30244, -84.37858 ..."
65,Walton,167,635,1010,3660,0.165347,0.173497,0.936257,0.356838,Walton,0.0,10.0,12.285714,1.592593,66.0,Walton,"POLYGON ((-86.03523 30.98825, -86.03532 30.982..."
66,Washington,68,113,210,649,0.323810,0.174114,3.158942,1.511111,Washington,0.0,3.0,4.285714,2.500000,67.0,Washington,"POLYGON ((-85.43543 30.56727, -85.46843 30.567..."


In [41]:
shpfile_name = "covid_14_metrics{}".format(yearDate)
print("Saving {}.shp in results/".format(shpfile_name))
gdf_merge_poly = gpd.GeoDataFrame(df_merge_poly_clean, geometry='geometry',crs="EPSG:4326")
gdf_merge_poly.to_file(driver = 'ESRI Shapefile', filename= os.path.join(result_folder,"{}.shp".format(shpfile_name)))

Saving covid_14_metrics111520.shp in results/


In [42]:
print("Saving {}.zip in publish/".format(shpfile_name))
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))   

Saving covid_14_metrics111520.zip in publish/
covid_14_metrics111520.dbf
covid_14_metrics111520.cpg
covid_14_metrics111520.shp
covid_14_metrics111520.shx
covid_14_metrics111520.prj


In [43]:
print(str(datetime.now()) + " Done with 6_COVID_14day_Metrics_Process")

2020-11-15 12:55:24.579670 Done with 6_COVID_14day_Metrics_Process
